In [43]:
import pandas as pd
import re
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [20]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mateg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('french')
stop_words.extend(['enfin','jusqu','part','peu','them','souvent','dedie','penser','presenton','eter','acc','esser','ainsi','non','partir','oeuvre','si','ete','cet', 'article', 'faire', 'bon', 'permettre', 'permet', 'proposer', 'proposons', 'obtenir', 'obtenons','tel', 'présenter', 'présentons', 'quel','face','mettre','basee','issu','gerer','but','etre','source','tre','pre','lie','sous', 'forme', 'faire', 'être', 'ete', 'donner', 'partie', 'étape', 'utiliser', 'utilisons', 'être', 'plus', 'fin', 'contexte'])

In [22]:
data = pd.read_csv("../data/Suicides_wikipedia_scrapping_fr.csv", encoding="utf-8-sig", delimiter=",")
data.head()

,Name,Link,Content
0,Adolfo_Couve,https://fr.wikipedia.org/wiki/Adolfo_Couve,"['id=""Biographie"">Biographie[modifier', '|', '..."
1,Boaz_Arad,https://fr.wikipedia.org/wiki/Boaz_Arad,"['id=""Biographie"">Biographie[modifier', '|', '..."
2,Ralph_Barton,https://fr.wikipedia.org/wiki/Ralph_Barton,"['id=""Biographie"">Biographie[modifier', '|', '..."
3,Mark_Campos,https://fr.wikipedia.org/wiki/Mark_Campos,"['id=""Biographie"">Biographie[modifier', '|', '..."
4,Michel_Caron_(artiste),https://fr.wikipedia.org/wiki/Michel_Caron_(ar...,"['id=""Biographie"">Biographie[modifier', '|', '..."


In [23]:
Biography = data.Content.values.tolist()
# Remove Emails
Biography = [re.sub('\S*@\S*\s?', '', sent) for sent in Biography]

# Remove new line characters
Biography = [re.sub('\s+', '', sent) for sent in Biography]

# Remove distracting single quotes
Biography = [re.sub("\'", "", sent) for sent in Biography]

# Remove the commas
Biography = [re.sub(",", " ", sent) for sent in Biography]

In [24]:
Biography[1]

'[id="Biographie">Biographie[modifier | modifier le code]Carrière artistique[modifier | modifier le code]Artiste multidisciplinaire (peinture  sculpture  photographie  vidéo)  il enseigne dans différentes écoles de Tel Aviv depuis 1993. Il est récompensé par le prix du Ministère de "lÉducation" et de la Culture (2003) et le prix du musée "dart" de Petah Tikva (2006)[2].Ses œuvres sont présentées dans différents musées  dont le musée "dIsraël" à Jérusalem[2].Mort[modifier | modifier le code]Le 1er février 2018  le site "dinformations" Mako révèle "quil" est visé par une enquête concernant des relations sexuelles "quil" aurait eues avec des élèves mineures. Le lendemain  il est retrouvé mort chez lui  apparemment suicidé[3].<span]'

In [25]:
Biography_clean_brackets = ['']*len(Biography)
for i in range(len(Biography)):
    for j in range(69, len(Biography[i]) - 7):
        Biography_clean_brackets[i]+=Biography[i][j]

In [26]:
Biography_clean_brackets[2]

' Kansas City (Missouri)  né "dun" père avocat et "dune" mère artiste-peintre  Ralph Barton se révèle très jeune doué pour le dessin.Il débute comme caricaturiste pour The Kansas City Star et The Kansas City Journal-Post  puis abandonne ses études  et part pour Chicago en 1909 tenter "lInstitut" "dart" de Chicago. Il renonce à vivre dans cette ville et revient à Kansas City pour se remarier avec Marie Jennings  sa première épouse.En 1910  il commence à contribuer au magazine satirique Puck ; il déménage à New York avec sa femme et leur fille  poursuit son travail pour Puck et "McCalls." Il loue ensuite un atelier "quil" partage avec le peintre Thomas Hart Benton "cest" Benton qui sert de premier modèle aux caricatures de Barton qui vont connaître un certain succès.En 1915  Puck décide "denvoyer" Barton en mission de reportage dans le Paris en guerre il "sattache" au pays  à la Ville lumière  et durant toute sa vie  y fera de fréquents séjours. Le 30 juin 1927  il est nommé chevalier de

In [27]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(Biography_clean_brackets))

In [28]:
data_words[1]

['stique',
 'modifier',
 'modifier',
 'le',
 'code',
 'artiste',
 'peinture',
 'sculpture',
 'photographie',
 'video',
 'il',
 'enseigne',
 'dans',
 'differentes',
 'ecoles',
 'de',
 'tel',
 'aviv',
 'depuis',
 'il',
 'est',
 'recompense',
 'par',
 'le',
 'prix',
 'du',
 'ministere',
 'de',
 'leducation',
 'et',
 'de',
 'la',
 'culture',
 'et',
 'le',
 'prix',
 'du',
 'musee',
 'dart',
 'de',
 'petah',
 'tikva',
 'ses',
 'œuvres',
 'sont',
 'presentees',
 'dans',
 'differents',
 'musees',
 'dont',
 'le',
 'musee',
 'disrael',
 'jerusalem',
 'mort',
 'modifier',
 'modifier',
 'le',
 'code',
 'le',
 'er',
 'fevrier',
 'le',
 'site',
 'dinformations',
 'mako',
 'revele',
 'quil',
 'est',
 'vise',
 'par',
 'une',
 'enquete',
 'concernant',
 'des',
 'relations',
 'sexuelles',
 'quil',
 'aurait',
 'eues',
 'avec',
 'des',
 'eleves',
 'mineures',
 'le',
 'lendemain',
 'il',
 'est',
 'retrouve',
 'mort',
 'chez',
 'lui',
 'apparemment',
 'suicide']

In [29]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['couve', 'braga', 'et', 'clemencia', 'rioseco', 'fernandez', 'adolfo', 'couve', 'est', 'laine', 'de', 'trois', 'freres', 'ne', 'valparaiso', 'en', 'il', 'passe', 'son', 'enfance', 'llay', 'llay', 'ou', 'il', 'habite', 'jusqua', 'lage', 'de', 'huit', 'ans', 'avant', 'de', 'sinstaller', 'avec', 'sa', 'famille', 'santiago', 'de', 'chile', 'il', 'acheve', 'sa', 'scolarite', 'au', 'colegio', 'san', 'ignacio', 'en', 'et', 'enterprend', 'ses', 'premieres', 'etudes', 'artistiques', 'lescuela', 'de', 'bellas', 'artes', 'de', 'luniversidad', 'de', 'chile', 'en', 'il', 'obtient', 'une', 'bourse', 'pour', 'etudier', 'lecole', 'des', 'beaux_arts', 'de', 'paris', 'et', 'lannee_suivante', 'poursuit', 'ses', 'etudes', 'aarts', 'student', 'league', 'new_york', 'il', 'ete', 'marie', 'avec', 'marta', 'carrasco', 'peintre', 'et', 'illustratrice', 'de', 'livres', 'pour', 'enfants', 'leur', 'fille', 'camila', 'couve', 'nee', 'en', 'fait', 'ses', 'debutes', 'dans', 'lecriture', 'en', 'en', 'publiant', 'esta

In [30]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

install spacy fr to run the below cell in anaconda as administrator : 
pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-2.0.0/fr_core_news_sm-2.0.0.tar.gz#egg=fr_core_news_sm==2.0.0
pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-2.0.0/fr_core_news_sm-2.0.0.tar.gz
python -m spacy download fr

In [34]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('fr', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(stop_words)
data_lemmatized = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['couver', 'braga', 'clemencia', 'rioseco', 'fernandez', 'adolfo', 'couv', 'lainer', 'frere', 'valparaiso', 'passer', 'enfance', 'llay', 'llay', 'habiter', 'jusqua', 'lage', 'an', 'sinstaller', 'famille', 'chile', 'acheve', 'scolarite', 'colegio', 'ignacio', 'enterprend', 'premier', 'etude', 'artistique', 'lescuela', 'bella', 'art', 'luniversidad', 'chile', 'bourse', 'etudier', 'lecole', 'beaux_art', 'poursuivre', 'etud', 'aarts', 'studer', 'leagu', 'new_york', 'marier', 'marta', 'carrasco', 'illustrateur', 'livr', 'enfant', 'fille', 'camila', 'couv', 'nee', 'debut', 'lecriture', 'publier', 'estamper', 'alfaguara', 'passer', 'vie', 'carthagene', 'petit', 'ville', 'province', 'saint', 'antoin', 'aprer', 'fort', 'depression', 'suicid']]


In [35]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1)]]


In [36]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aarts', 1),
  ('acheve', 1),
  ('adolfo', 1),
  ('alfaguara', 1),
  ('an', 1),
  ('antoin', 1),
  ('aprer', 1),
  ('art', 1),
  ('artistique', 1),
  ('beaux_art', 1),
  ('bella', 1),
  ('bourse', 1),
  ('braga', 1),
  ('camila', 1),
  ('carrasco', 1),
  ('carthagene', 1),
  ('chile', 2),
  ('clemencia', 1),
  ('colegio', 1),
  ('couv', 2),
  ('couver', 1),
  ('debut', 1),
  ('depression', 1),
  ('enfance', 1),
  ('enfant', 1),
  ('enterprend', 1),
  ('estamper', 1),
  ('etud', 1),
  ('etude', 1),
  ('etudier', 1),
  ('famille', 1),
  ('fernandez', 1),
  ('fille', 1),
  ('fort', 1),
  ('frere', 1),
  ('habiter', 1),
  ('ignacio', 1),
  ('illustrateur', 1),
  ('jusqua', 1),
  ('lage', 1),
  ('lainer', 1),
  ('leagu', 1),
  ('lecole', 1),
  ('lecriture', 1),
  ('lescuela', 1),
  ('livr', 1),
  ('llay', 2),
  ('luniversidad', 1),
  ('marier', 1),
  ('marta', 1),
  ('nee', 1),
  ('new_york', 1),
  ('passer', 2),
  ('petit', 1),
  ('poursuivre', 1),
  ('premier', 1),
  ('province', 1),
 

In [46]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=33,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

...

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

In [47]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"modifier" + 0.012*"mourir" + 0.011*"coder" + 0.010*"an" + '
  '0.010*"apr" + 0.009*"alors" + 0.007*"nouveau" + 0.006*"premier" + '
  '0.006*"prendre" + 0.005*"quil"'),
 (1,
  '0.009*"swartz" + 0.009*"scott" + 0.007*"john" + 0.006*"william" + '
  '0.006*"assassin" + 0.005*"poisson" + 0.005*"londr" + 0.005*"londre" + '
  '0.004*"seigneur" + 0.004*"americain"'),
 (2,
  '0.020*"nobunaga" + 0.012*"politique" + 0.007*"gouvernement" + '
  '0.007*"membre" + 0.007*"policer" + 0.006*"service" + 0.005*"seppuku" + '
  '0.005*"attaquer" + 0.005*"peiner" + 0.005*"grouper"'),
 (3,
  '0.017*"hideyoshi" + 0.010*"hidetsugu" + 0.009*"batailler" + 0.008*"loi" + '
  '0.008*"roi" + 0.007*"vassal" + 0.007*"toyotomi" + 0.006*"tiber" + '
  '0.005*"cleomene_ier" + 0.005*"trajan"'),
 (4,
  '0.009*"laisser" + 0.008*"chef" + 0.007*"turing" + 0.007*"histoire" + '
  '0.006*"deuxieme" + 0.006*"publier" + 0.006*"artiste" + 0.006*"art" + '
  '0.005*"lendemain" + 0.004*"robert"'),
 (5,
  '0.009*"grand" + 

In [48]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

...

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\mateg\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(


Perplexity:  -10.124418512651424

Coherence Score:  0.36530605116796977


In [49]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\mateg\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.360516  0.029536       1        1  36.416061
5      0.195428 -0.145206       2        1  15.796939
7      0.017348  0.267195       3        1  10.439946
6     -0.006680 -0.012402       4        1   8.701833
2     -0.094708  0.123165       5        1   6.770007
4     -0.088115 -0.057188       6        1   6.235438
9     -0.051207 -0.049953       7        1   5.883294
3     -0.111337 -0.047026       8        1   3.266293
8     -0.118166 -0.046278       9        1   3.259329
1     -0.103078 -0.061842      10        1   3.230862, topic_info=      Category         Freq             Term        Total  loglift  logprob
86     Default  2797.000000         modifier  2797.000000  30.0000  30.0000
650    Default   622.000000             film   622.000000  29.0000  29.0000
87     Default  1644.000000           mourir  1644.000000  28.0000  28.0000
45644  Default   477.000000         nobunaga   477.000000  27.0000  27.0000
70     Default  1495.000000            coder  1495.000000  26.0000  26.0000
44474  Default   438.000000       formiggini   438.000000  25.0000  25.0000
1520   Default   399.000000        militaire   399.000000  24.0000  24.0000
409    Default  1134.000000            grand  1134.000000  23.0000  23.0000
190    Default   592.000000           guerre   592.000000  22.0000  22.0000
17055  Default   270.000000             clan   270.000000  21.0000  21.0000
488    Default  1278.000000            alors  1278.000000  20.0000  20.0000
9912   Default   351.000000        batailler   351.000000  19.0000  19.0000
4917   Default   347.000000        politique   347.000000  18.0000  18.0000
112    Default   357.000000         allemand   357.000000  17.0000  17.0000
895    Default   408.000000           membre   408.000000  16.0000  16.0000
1436   Default   362.000000          general   362.000000  15.0000  15.0000
235    Default   381.000000           nommer   381.000000  14.0000  14.0000
4491   Default   378.000000          chateau   378.000000  13.0000  13.0000
55     Default  1301.000000          premier  1301.000000  12.0000  12.0000
494    Default   277.000000             chef   277.000000  11.0000  11.0000
45313  Default   193.000000        hideyoshi   193.000000  10.0000  10.0000
1515   Default   410.000000          janvier   410.000000   9.0000   9.0000
96     Default   683.000000             quil   683.000000   8.0000   8.0000
4      Default  1512.000000               an  1512.000000   7.0000   7.0000
356    Default  1481.000000              apr  1481.000000   6.0000   6.0000
3710   Default   356.000000             ruer   356.000000   5.0000   5.0000
30     Default   650.000000          famille   650.000000   4.0000   4.0000
1349   Default   275.000000       professeur   275.000000   3.0000   3.0000
57     Default   267.000000          publier   267.000000   2.0000   2.0000
511    Default   578.000000            ecrit   578.000000   1.0000   1.0000
...        ...          ...              ...          ...      ...      ...
39211  Topic10    39.154118            nitti    40.253551   3.4047  -5.6833
12260  Topic10    28.267286           street    29.151464   3.4016  -6.0091
32116  Topic10    24.463337            aaron    25.347940   3.3969  -6.1536
31735  Topic10    24.454573         hartford    25.344471   3.3967  -6.1540
6098   Topic10    24.002707             lord    24.886951   3.3962  -6.1726
1430   Topic10    24.018251              det    24.904633   3.3962  -6.1720
43848  Topic10   100.802742           swartz   104.650200   3.3950  -4.7376
6646   Topic10    23.068150         columbia    23.953300   3.3948  -6.2123
6103   Topic10    22.825983               of    23.710033   3.3944  -6.2229
18507  Topic10    22.798481        chercheur    23.682650   3.3944  -6.2241
1369   Topic10    22.232880           credit    23.117792   3.3934  -6.2492
5652   Topic10    21.916155          past

In [50]:
pyLDAvis.save_html(vis, 'lda.html')